In [1]:
# 조원분들거 참조
import requests
from urllib.parse import quote, unquote, urlencode  # 한글을 URLencode 변환하는 함수. 롯데 --> '%EB%A1%AF%EB%8D%B0'
import json
from urllib.request import urlopen
import datetime
from datetime import datetime as dt, timedelta as td
import pymysql

def getDataPortalSearch(base_date, base_time):
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
    queryString = "?" + urlencode(
            {
                'serviceKey' : 's86nUoT8OvF9KjCQEnAYi6kAQ56CU5iiqDHjh384K4gzAVzXj4qFqiCulxZJuhz9yfgwb87yUG/CmL1hD5RO+g==', 
                'pageNo' : '1',
                'numOfRows' : '1000',
                'dataType' : 'JSON',
                'base_date' : base_date,
                'base_time' : base_time,
                'nx' : '98',
                'ny' : '76'
            }
        )

    total_url = url + queryString
    response = requests.get(total_url, verify=False)
    return response.text

def weatherData():   # response 데이터 가져와서 사용하기 좋게 가공
    
    # 검색 날짜, 시간 설정
    baseDate = f'{dt.now().date().strftime("%Y%m%d")}'
    # nowTime = int(dt.now().time().strftime("%H"))
    nowTime = dt.now().time()
    API_TIME = [datetime.time(2,10), datetime.time(5,10), datetime.time(8,10),
                datetime.time(11,10), datetime.time(14,10), datetime.time(17,10),
                datetime.time(20,10), datetime.time(23,10)]

    for i, time in enumerate(API_TIME[::-1]):
        if nowTime >= time:
            baseTime = f'{time.strftime("%H")}00'
        else:
            baseDate = f'{(dt.now().date() - td(days=1)).strftime("%Y%m%d")}'
            baseTime = f'{API_TIME[i-1].strftime("%H")}00'

    try:
        result = getDataPortalSearch(baseDate, baseTime)
        json_data = json.loads(result)
        weather_data = json_data['response']['body']['items']['item']
    except Exception as e:
        print('찾는 데이터가 없습니다.')
    
    changeDate = weather_data[0]['fcstDate']
    changeTime = weather_data[0]['fcstTime']
    fcstData = dict()
    allFcstData = []

    for item in weather_data:
        del item['baseDate'], item['baseTime'], item['nx'], item['ny']
        if item['category'] == 'TMP':   # 온도
            item['tmp'] = item['fcstValue']
        elif item['category'] == 'VEC': # 풍향
            item['vec'] = item['fcstValue']
        elif item['category'] == 'WSD': # 풍속
            item['wsd'] = item['fcstValue']
        elif item['category'] == 'SKY': # 하늘 상태
            item['sky'] = item['fcstValue']
        elif item['category'] == 'PTY': # 강수 형태 
            item['wsd'] = item['fcstValue']
        elif item['category'] == 'POP': # 강수 확률
            item['pop'] = item['fcstValue']
        elif item['category'] == 'PCP': # 1시간 강수량
            item['pcp'] = item['fcstValue']
        elif item['category'] == 'REH': # 습도
            item['reh'] = item['fcstValue']
        elif item['category'] == 'SNO': # 신적설
            item['sno'] = item['fcstValue']
        elif item['category'] == 'TMN': # 하루 최저 온도
            item['tmn'] = item['fcstValue']
        elif item['category'] == 'TMM': # 하루 최고 온도
            item['tmm'] = item['fcstValue']
        del item['category'], item['fcstValue']

        if changeDate == item['fcstDate'] and changeTime == item['fcstTime']:
            fcstData = fcstData|item
        else:
            allFcstData.append(fcstData)
            fcstData = dict()
            fcstData = item

        changeDate = item['fcstDate']
        changeTime = item['fcstTime']

    return allFcstData

def insertDB():
    # 주현집 host
    # conn = pymysql.connect(host='localhost', user = 'root', password='12345',
    #                        db = 'miniproject', charset='utf8')
    # 성현dDB
    conn = pymysql.connect(host='210.119.12.66', user = 'root', password='12345',
                           db = 'miniproject01', charset='utf8')
    cur = conn.cursor()    # Connection으로부터 Cursor 생성

    allFcstData = weatherData()

    for fcstData in allFcstData:
        if 'tmn' in fcstData.keys():
            query = '''INSERT INTO weather (fcstDate, fcstTime, TMP, VEC, WSD, SKY, POP, PCP, REH, SNO, TMN)
                      VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
            cur.execute(query, (fcstData['fcstDate'], fcstData['fcstTime'], fcstData['tmp'],
                            fcstData['vec'], fcstData['wsd'], fcstData['sky'], fcstData['pop'],
                            fcstData['pcp'], fcstData['reh'], fcstData['sno'], fcstData['tmn']))
        elif 'tmm' in fcstData.keys():
            query = '''INSERT INTO weather (fcstDate, fcstTime, TMP, VEC, WSD, SKY, POP, PCP, REH, SNO, TMM)
                      VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
            cur.execute(query, (fcstData['fcstDate'], fcstData['fcstTime'], fcstData['tmp'],
                            fcstData['vec'], fcstData['wsd'], fcstData['sky'], fcstData['pop'],
                            fcstData['pcp'], fcstData['reh'], fcstData['sno'], fcstData['tmm']))
        else:
            query = '''INSERT INTO weather (fcstDate, fcstTime, TMP, VEC, WSD,  SKY, POP, PCP, REH, SNO)
                      VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
            cur.execute(query, (fcstData['fcstDate'], fcstData['fcstTime'], fcstData['tmp'],
                            fcstData['vec'], fcstData['wsd'], fcstData['sky'], fcstData['pop'],
                            fcstData['pcp'], fcstData['reh'], fcstData['sno']))
            
        conn.commit()

    conn.close()
    print('저장')

insertDB()

{"response":{"header":{"resultCode":"10","resultMsg":"최근 1일 간의 자료만 제공합니다."}}}
